# **Find the Best Location to Open an Ice Cream Shop in Boston Area**
Capstone Project
***

# Introduction
Selecting a location for a business is one of the most critical decisions in running a business. Business decision-makers need to consider different factors in finding the right location for the business, such as financial factors, market factors, and the factors that will affect their demand and revenue. This project can help those who plan to open a new business such as an ice-cream shop and need to find the best location for the shop in a city.

# Business Problem
Finding the best location to start a new business in a large city such as Boston is challenging. Boston has 23 neighborhoods with various places that attract tourists and people to visit. Neighborhoods of Boston are different in terms of factors that can directly or indirectly affect business success. For an informed decision, it is essential to evaluate all neighborhoods based on the factors that affect running a successful business, such as the number of competitors and the potential demand in each neighborhood. Budget limitations in renting or buying a place also play an essential role in this decision-making, especially for small businesses.  

In this project, We want to help a client find the best neighborhood in Boston to open an Ice Cream shop. Finding a neighborhood that is close enough to some amenities and venues, has fewer competitors, and is affordable is challenging for our client. An ice cream shop is good near a place where many people visit, such as near a cinema, park, garden, playground, etc. Also, the client has a limited budget, and a neighborhood with fewer competitors is less risky for him/her to start the business. 

# Data
## Use BeautifulSoup to extract the neighborhoods:
We need to extract the 23 neighborhoods of Boston and the median one-bedroom rent price (as an estimate of the renting cost of the shop) of each neighborhood. For this purpose, We will use the following website and apply the BeautifulSoup website scraping library to extract the data.
 http://realestate.boston.com/renting/2019/02/19/median-one-bedroom-rent-price/


In [1]:
import numpy as np
import pandas as pd

from urllib.request import urlopen
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'bs4'

In [ ]:
url = "http://realestate.boston.com/renting/2019/02/19/median-one-bedroom-rent-price/"
html = urlopen(url)

# create a Beautiful Soup object from the html
soup = BeautifulSoup(html, 'lxml')
type(soup)

# using right click and inspect on the webpage we found the class of the useful part of the page:
website_text = soup.find('div', {'class' :'content-text'}).text

#convert the string to list
import re
My_list= re.findall('(.+?)\n' , website_text)
My_list


Transform the data into pandas dataframe:

In [ ]:
#remove the first 3 unuseful elements of the list
del My_list[0:3]

#create a list of all the neighborhoods:
My_nb= My_list[0::3]

#add ', MA, USA' to the name of the neighborhoods to make sure geopy we will find correct locations
nb=[s + ', MA, USA' for s in My_nb]

#create a list of average rent of the neighborhoods
rt=My_list[1::3]

#to remove the string 'Winter 2019: $' next to the rent values
My_list2=[s.replace('Winter 2019: $', '') for s in rt]

#create a panda dataframe of neighborhoods and their average rent of one-be apartment 
df_boston = pd.DataFrame({'Neighborhood':nb,'Average Rent':My_list2})

#to make sure we will find the right location of the neighborhoods in boston area
df_boston.xs(8)['Neighborhood']= 'Boston Downtown, MA,USA'
df_boston.xs(21)['Neighborhood']= 'South End Boston, MA,USA'

#remove 'Leather District' as geopy cannot fint its location
df_boston.drop([13], inplace=True)
df_boston

In [ ]:
print('We are investigating {} neighborhoods of Boston.'.format (len(df_boston)))

## Use GeopPy to get the coordinates of the neighborhoods:
Use the GeoPy Python package to get the latitude and the longitude coordinates of all the neighborhoods of Boston:

In [ ]:
#!conda install -c conda-forge geopy --yes 

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
latlon = df_boston.Neighborhood.apply(lambda addr: geolocator.geocode(addr))
df_boston["Latitude"] = [x.latitude for x in latlon]
df_boston["Longitude"] = [x.longitude for x in latlon]

df_boston.to_csv('df-boston.csv')#save the dataframe
df_boston

## Map the neighborhoods:

In [ ]:
#get location of Boston to map the neighborhoods
address = 'BOSTON, MA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Boston are {}, {}.'.format(latitude, longitude))


In [ ]:
#!conda install -c conda-forge folium=0.5.0 --yes 

In [ ]:
# create map of Boston using latitude and longitude values of the neighborhoods
import folium 
map_Boston = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_boston['Latitude'], df_boston['Longitude'], df_boston['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Boston)  
            
map_Boston

## Use Foursquare API to get information about some venues around these neighborhoods:

Looking for Movie Theater, Playground, Park, Garden, Water Park, General Entertainment, Stadium, Amphitheater, Aquarium, Street Art, Beach, Recreation Center, and Pedestrian Plaza venues in walking distance (500 meters) from each of these neighborhood. These venues are places that many people usually visit them for entertainment and hence we will have good demand for ice cream around them. 

In [ ]:
# Venue categories to search around each neighborhood
# The list of all venue categories of FourSquare can be found in the following link: https://developer.foursquare.com/docs/resources/categories. 

fs_categories = {
    
    'Movie Theater': '4bf58dd8d48988d17f941735',
    'Playground': '4bf58dd8d48988d1e7941735',
    'Park': '4bf58dd8d48988d163941735',
    'Garden': '4bf58dd8d48988d15a941735',
    'Water Park': '4bf58dd8d48988d193941735',
    'General Entertainment': '4bf58dd8d48988d1f1931735',
    'Stadium' : '4bf58dd8d48988d184941735',
    'Amphitheater' :'56aa371be4b08b9a8d5734db',
    'Aquarium' : '4fceea171983d5d06c3e9823',
    'Street Art' : '52e81612bcbc57f1066b79ee',
    'Beach' : '4bf58dd8d48988d1e2941735',
    'Recreation Center' : '52e81612bcbc57f1066b7a26',
    'Pedestrian Plaza': '52e81612bcbc57f1066b7a25'
}

', ' .join([ cat for cat in fs_categories])

In [ ]:
# Install FourSquare client library
!pip install foursquare

In [ ]:
import foursquare
CLIENT_ID = '1HABFONPEEOMT0AS2GVXPK53A4EO43AXRTJVN3B03CG3N3BE'
CLIENT_SECRET = '0CR2ZL5CZHLX5OSIIJRZCBPXGCC1TIS2HQ04CUYLFBFRBG1Q' 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
fs = foursquare.Foursquare( CLIENT_ID, CLIENT_SECRET)
RADIUS = 500 

In [ ]:
# Define a function to search nearby venues for the categories specified and convert the result as dataframe
def venues_nearby(latitude, longitude, category, verbose=True):    
    results = fs.venues.search(
        params = {
            'query': category, 
            'll': '{},{}'.format(latitude, longitude),
            'radius': RADIUS,
            'categoryId': fs_categories[category]
        }
    )    
    df = json_normalize(results['venues'])
    cols = ['Name','Latitude','Longitude']    
    if( len(df) == 0 ):        
        df = pd.DataFrame(columns=cols)
    else:        
        df = df[['name','location.lat','location.lng']]
        df.columns = cols
    if( verbose ):
        print('{} "{}" venues are found within {}m of location'.format(len(df), category, RADIUS))
    return df

In [ ]:
# to check the spesified venue categories for all the neighborhoods
from pathlib import Path

venues_csv = Path('./boston_venues.csv')
df_venues = None

# check the venues data is explored and downloaded 
if( venues_csv.exists() ):
    df_venues = pd.read_csv('./boston_venues.csv')
else:    
    # construct a dataframe to store data
    df_venues = pd.DataFrame(columns=[ 'Category','Latitude', 'Longitude'])
    for (name, average_rent, latitude, longitude) in df_boston.itertuples(index=False):
        for cat, cat_id in fs_categories.items():
            df = venues_nearby(latitude, longitude, cat, verbose=False)
            df['Neighborhood'] = name
            df['Average Rent'] = average_rent
            df['Category'] = cat
            df_venues = df_venues.append(df, sort=True)
    df_venues.to_csv('boston_venues.csv', index=False)

In [ ]:
print('In total, {} venues are found in Boston.'.format(len(df_venues)))

df_venues.head()

In [ ]:
#number of venues from each category
import matplotlib.pyplot as plt
p= df_venues['Category'].value_counts().to_frame(name='Total Count')
p.index.name = 'Venue Category'
p.plot.bar(y='Total Count')

In [ ]:
# number of venues for each neighborhood
df_groupby= df_venues.groupby('Neighborhood').count().reset_index()
df_groupby

In [ ]:
#the dataset of all the venues from different categories in each neighborhood
boston_onehot = pd.get_dummies(df_venues[['Category']], prefix="", prefix_sep="")#one hot encoding
boston_onehot['Neighborhood'] = df_venues['Neighborhood'] 
fixed_columns = [boston_onehot.columns[-1]] + list(boston_onehot.columns[:-1])
boston_onehot = boston_onehot[fixed_columns]
boston_onehot.head()

## Group rows by neighborhood to compare the frequency of different venue categories in each neighborhood: 

In [ ]:
boston_grouped = boston_onehot.groupby('Neighborhood').sum().reset_index()
boston_grouped

In [ ]:
#neighborhoods without any 'Movie Theater, Playground, Park, Garden, Zoo, Water Park, General Entertainment, Stadium'venues: 
print('We will remove {} from the investigation as does not have any Movie Theater, Playground, Park, Garden, Water Park, General Entertainment, Stadium, Amphitheater, Aquarium, Street Art, Beach, Recreation Center, Pedestrian Plaza venues around.'
      .format(set(df_boston['Neighborhood']).difference(boston_grouped['Neighborhood'])))

In [ ]:
### sort venues for each neighborhood
for hood in boston_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = boston_grouped[boston_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(int)
    temp = temp.round({'freq': 0})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True))
    print('\n')

In [ ]:
#a function to sort the venues in decrising order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
#find top 5 most common venues around each neighborhood
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues

columns =[ 'Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)

neighborhoods_venues_sorted['Neighborhood'] = boston_grouped['Neighborhood']


for ind in np.arange(boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(boston_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted['Average Rent'] = df_boston_nj['Average Rent']
neighborhoods_venues_sorted['# of venues']= df_groupby['Category']

neighborhoods_venues_sorted

## Looking for Ice cream shops within walking distance (500 meters) of each of the neighborhoods to get an understanding of the competitors in each neighborhood:

In [ ]:
# Ice cream shop category in FourSquare
fs_categories = {
    'Ice Cream Shop': ' 4bf58dd8d48988d1c9941735',
}

In [ ]:
# check Ice cream shop venue for all the neighborhoods
from pathlib import Path

venues_csv = Path('./boston_ic_cream.csv')
df_venues = None
if( venues_csv.exists() ):
    df_venues = pd.read_csv('./boston_ic_creams.csv')
else:    
    # construct a dataframe to store data
    df_venues = pd.DataFrame(columns=[ 'Category','Latitude', 'Longitude'])
    for (name, average_rent, latitude, longitude) in df_boston.itertuples(index=False):
        for cat, cat_id in fs_categories.items():
            df = venues_nearby(latitude, longitude, cat, verbose=False)
            df['Neighborhood'] = name
            df['Average Rent'] = average_rent
            df['Category'] = cat
            df_venues = df_venues.append(df, sort=True)
    df_venues.to_csv('boston__ic_cream.csv', index=False)
   
print('Total {} Ice Cream Shop are found in Boston Area.'.format(len(df_venues)))

In [ ]:
#number of Ice cream shops within walking distance of each of the neighborhoods
boston_onehot_ic = pd.get_dummies(df_venues[['Category']], prefix="", prefix_sep="")#one hot encoding
boston_onehot_ic['Neighborhood'] = df_venues['Neighborhood'] # add neighborhood column back to dataframe
fixed_columns = [boston_onehot_ic.columns[-1]] + list(boston_onehot_ic.columns[:-1])
boston_onehot_ic = boston_onehot_ic[fixed_columns]
boston_grouped_ic = boston_onehot_ic.groupby('Neighborhood').sum().reset_index()
boston_icecreamshops = pd.merge(df_boston, boston_grouped_ic, on=['Neighborhood'], how='outer').fillna(0)
boston_icecreamshops 

In [ ]:
#add the number of ice cream shops to the dataframe of all the neighborhoods with the list of 5 most common venues
df_neighborhoods = pd.merge(neighborhoods_venues_sorted, boston_icecreamshops, on=['Neighborhood'], how='inner')
df_neighborhoods

# Methodology

To start a new business, we need to select neighborhoods with fewer competitors. For this purpose, we will remove the neighborhoods with more than one ice cream shops. We will use K-means clustering to classify the neighborhoods based on the nearby venues and their frequencies (except the ice cream shop category). To make the decision, we will sort the neighborhoods in each cluster based on their average rent price. A neighborhood with more number of venues, no ice cream shop, and a lower price will be the best option.

## Select neighborhoods with fewer competitors
We remove neighborhoods with more than one ice cream shop:

In [ ]:
low_competitor=df_neighborhoods[df_neighborhoods['Ice Cream Shop'] < 2]
low_competitor

## Cluster the neighborhoods: 
Run k-means to cluster the neighborhood into 5 clusters based on the frequency of different venue categories (except ice-cream shop) around them

In [ ]:
# different categories for neighborhoods with less than 2 ice-cream shops
boston_grouped_reduced= boston_grouped[boston_grouped.index.isin(low_competitor.index)]
boston_grouped_reduced

In [ ]:
from sklearn.cluster import KMeans
kclusters = 5 # set number of clusters

boston_grouped_clustering = boston_grouped_reduced.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(boston_grouped_clustering)

# find cluster labels generated for each row of boston_grouped dataframe (neghborhoods of boston excep Jamaica Plain and Mattapan)
kmeans.labels_[0:15] 

In [ ]:
#to add the clusters to the dataframe
low_competitor.insert(0, 'Cluster Label', kmeans.labels_)# add clustering labels

In [ ]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
boston_merged = pd.merge(df_boston, low_competitor, on=['Neighborhood', 'Average Rent','Latitude','Longitude'], how='inner')
boston_merged

In [ ]:
#!conda install -c conda-forge folium=0.5.0 --yes 

## Map the clustered neighborhoods

In [ ]:
#visualize the clustered neighborhoods of boston
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Matplotlib and associated plotting modules
import folium 
from folium.features import CustomIcon
from folium.plugins import MarkerCluster
import matplotlib.cm as cm
import matplotlib.colors as colors

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(boston_merged['Latitude'], boston_merged['Longitude'], boston_merged['Neighborhood'], boston_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Results
The result of 5 clusters are specified in this section. The neighborhoods in each cluster are sorted based on their average rent price:

 **Cluster 1:**

In [ ]:
cluster1= boston_merged.loc[boston_merged['Cluster Label'] == 0]
cols = [0,1,5,6,7,8,9,10,11]
cluster1[cluster1.columns[cols]]

 **Cluster 2:**

In [ ]:
cluster2=boston_merged.loc[boston_merged['Cluster Label'] == 1]
cluster2[cluster2.columns[cols]].sort_values(['Average Rent'], ascending=True) #sort the neighborhoods in the cluster based on their average rent

 **Cluster 3:**

In [ ]:
cluster3=boston_merged.loc[boston_merged['Cluster Label'] == 2]
cluster3[cluster3.columns[cols]].sort_values(['Average Rent'], ascending=True) #sort the neighborhoods in the cluster based on their average rent

**Cluster 4:**

In [ ]:
cluster4=boston_merged.loc[boston_merged['Cluster Label'] == 3]
cluster4[cluster4.columns[cols]].sort_values(['Average Rent'], ascending=True)  #sort the neighborhoods in the cluster based on their average rent

**Cluster 5:**

In [ ]:
cluster5=boston_merged.loc[boston_merged['Cluster Label'] == 4]
cluster5[cluster5.columns[cols]].sort_values(['Average Rent'], ascending=True)  #sort the neighborhoods in the cluster based on their average rent

# Discussion

According to the result of our analysis, South End Boston (in cluster 1) is the best neighborhood to open an ice cream shop in Boston. It has the most number of total venues and is expected to attract many people to visit. This neighborhood with 12 parks, 7 gardens, 4 playgrounds, and one movie theater and no ice cream shop within walking distance is the best option. However, the final decision is dependent on the budget of the client to rent the place. According to the budget limitations of the client, we can find other neighborhoods and select the one with the most number of venues and less number of ice cream shops around. This clustering can also help our client compare different neighborhoods to make a better decision. For example, South End Boston is a better option comparing to both neighborhoods in cluster 3 as has less average rent price, a larger number of venues, and no ice cream shop around. 


# Conclusion

We used the information about the neighborhoods of Boston to help my client to select the best location to open an ice cream shop in Boston. We considered number of competitors, demand for ice cream, and budget limitations to find the best location options. 

This project has some limitations. First, we have limited the location options to the neighborhoods of Boston, however considering more specific locations such as particular streets can improve the accuracy of our decision. Second, we used the median of one bedroom rent price in each neighborhood as an estimate of the rent price. Third, we limited our investigation for the venues within 500 meter of the neighborhoods and changing it can affect the clusters. Also, we considered equal weights for different venues and as an extension one can consider different weights for the venues (for example being near a beach has more impact on the demand for ice cream comparing to being near a pedestrian plaza).